## This is automated visa application processing system using AI.

##### This project will use multi AI model which will assist visually impaired person in filling Visa. Capture his image and then transform into EVisa


In [54]:
import os
import json
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI



In [55]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


    
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [12]:
#

from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
system_message = "You are a helpful assistant for an E Visa application called Eazee. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [63]:
import gradio as gr
from fpdf import FPDF
import cv2
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables and OpenAI key
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)
MODEL = "gpt-4o-mini"

def openai_chat(message, history):
    # Prepare messages for OpenAI API
    messages = [{"role": "system", "content": system_message}]
    for user, assistant in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    reply = response.choices[0].message.content
    # Simple intent detection for visa
    show_form = "visa" in message.lower()
    history.append((message, reply))
    return history, gr.update(visible=show_form)

def create_visa_application_pdf(firstname, lastname, address, gender, image):
    img_path = "visa_photo.jpg"
    if image is not None:
        cv2.imwrite(img_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    else:
        return None
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=14)
    pdf.cell(200, 10, "E-Visa Application Form", ln=True, align="C")
    pdf.ln(10)
    pdf.cell(50, 10, f"First Name: {firstname}", ln=True)
    pdf.cell(50, 10, f"Last Name: {lastname}", ln=True)
    pdf.cell(50, 10, f"Gender: {gender}", ln=True)
    pdf.multi_cell(0, 10, f"Address: {address}", align="L")
    pdf.ln(10)
    pdf.cell(50, 10, "Applicant Photo:", ln=True)
    if os.path.exists(img_path):
        pdf.image(img_path, x=pdf.get_x(), y=pdf.get_y(), w=40)
    pdf_path = "visa_application.pdf"
    pdf.output(pdf_path)
    return pdf_path

with gr.Blocks() as demo:
    gr.Markdown("## Eazee: AI-Powered E-Visa Assistant")
    chat_history = gr.State([])
    with gr.Row():
        chatbox = gr.Chatbot()
    with gr.Row():
        user_msg = gr.Textbox(label="Type your message")
        send_btn = gr.Button("Send")
    visa_form = gr.Group(visible=False)
    with visa_form:
        gr.Markdown("### E-Visa Application Form")
        with gr.Row():
            firstname = gr.Textbox(label="First Name")
            lastname = gr.Textbox(label="Last Name")
        address = gr.Textbox(label="Address")
        gender = gr.Radio(["Male", "Female", "Other"], label="Gender")
        image = gr.Image(label="Take your photo", sources=["webcam"])
        pdf_file = gr.File(label="Download Visa Application PDF")
        btn = gr.Button("Generate PDF")
        btn.click(
            create_visa_application_pdf,
            inputs=[firstname, lastname, address, gender, image],
            outputs=pdf_file
        )
    send_btn.click(
        openai_chat,
        inputs=[user_msg, chat_history],
        outputs=[chatbox, visa_form]
    )

demo.launch()

/var/folders/kw/_0591_px5xnc85wlsh08blbm0000gn/T/ipykernel_82929/1454100266.py:55: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


In [58]:
Visa_function = {
    "name": "create_visa_application_pdf",
    "description": (
        "Generate a visa application PDF. "
        "Call this whenever you need to create a visa application, for example when a customer asks 'Can you help with my visa application?'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "firstname": {
                "type": "string",
                "description": "The applicant's first name"
            },
            "lastname": {
                "type": "string",
                "description": "The applicant's last name"
            },
            "address": {
                "type": "string",
                "description": "The applicant's address"
            },
            "gender": {
                "type": "string",
                "description": "The applicant's gender"
            },
            "photo": {
                "type": "string",
                "description": "Base64-encoded image of the applicant's photo"
            }
        },
        "required": [
            "firstname", "lastname", "address", "gender", "photo"
            
        ],
        "additionalProperties": False
    }
}

In [ ]:
visa_tools = [{"type": "function", "function": Visa_function}]
#hotel_tools = [{"type": "function", "function": Hotel_price_function}]
tools = visa_tools 

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [61]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

def chat_fn(message, history, show_form):
    # Simple intent detection for demo purposes
    if "visa" in message.lower() and ("apply" in message.lower() or "create" in message.lower()):
        reply = "Sure, please fill out the visa application form below."
        show_form = True
    else:
        reply = "How can I assist you?"
        show_form = False
    history.append((message, reply))
    return history, gr.update(visible=show_form)

with gr.Blocks() as demo:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        state = gr.State(False)  # show_form state
    with gr.Row():
        msg = gr.Textbox(label="Type your message")
        send = gr.Button("Send")
    visa_form = gr.Group(visible=False)
    with visa_form:
        gr.Markdown("## E-Visa Application Form")
        with gr.Row():
            firstname = gr.Textbox(label="First Name")
            lastname = gr.Textbox(label="Last Name")
        address = gr.Textbox(label="Address")
        gender = gr.Radio(["Male", "Female", "Other"], label="Gender")
        image = gr.Image(label="Take your photo", sources=["webcam"])
        pdf_file = gr.File(label="Download Visa Application PDF")
        btn = gr.Button("Generate PDF")
        btn.click(
            create_visa_application_pdf,
            inputs=[firstname, lastname, address, gender, image],
            outputs=pdf_file
        )
    send.click(
        chat_fn,
        inputs=[msg, chatbot, state],
        outputs=[chatbot, visa_form]
    )
demo.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
